# __Report Lab1__
---
>## __Overview :__
>วัตถุประสงค์คือออกแบบระบบโดยใช้ ROS2 สำหรับสร้างระบบ Random Walk ที่มีสัญญาณรบกวนที่เป็นรูปแบบ Random Noise ที่มีการกระจายตัวแบบ Gaussian Distribution 
>(การกระจายตัวนี้จะประกอบไปด้วยค่า mean และ variance) โดยการใช้ turtlesim ในการแสดงผลของระบบ โดยที่จะต้องมี service สำหรับการกำหนดค่า mean และ variance ของรูปแบบ Gaussian Distribution และ มีการสร้าง launch file สำหรับการรันโปรแกรมทั้งหมด และจะต้องมี node สำหรับสร้าง Noise 2 node โดยมีการแยก namespace และ node สำหรับการรวมสัญญาณไปให้กับ turtlesim และแสดงผลออกมาในรูปแบบของการเคลื่อนที่ของตัวเต่า
>
>เราทำได้แบ่งออกเป็น 4 ส่วน คือ



<p align="center">
<img src="pic1.png" width="90%">
</p>

<p align="center">
ภาพที่ 1 ภาพรวมของระบบ
</p>

>## Lab1 Interface :
>### overview :
>>    เราต้องการสร้าง data type service ที่มีการส่งค่า mean และ variance มาจาก client และ return empty กลับไปยัง client โดยที่เราแยก pkg ออกมาสำหรับการสร้าง data type และ service นี้ โดยเฉพาะ 
>>
>> โดย pkg นี้จะมีไฟล์ SetNoise.srv ที่เป็นชื่อ data type service และไฟล์ Cmake ที่มีการใส่ code ในการเจน date type นี้
>### srv file :
>>File : "lab1_interfaces/srv/SetNoise.srv"
>>```
>>float64 mean
>>float64 variance
>>--- 
>>```
>
>
>### cmakelist :
>เพื่อที่จะสามารถ Build package ได้เราจึงแก้ไขไฟล์ CMakeLists.txt
>>File : "lab1_interfaces/CMakeLists.txt"
>>```cmake
>>rosidl_generate_interfaces(${PROJECT_NAME}
>>  "srv/SetNoise.srv"
>>  DEPENDENCIES std_msgs
>>)
>>```

>## Noise Generator :
>### overview :
>
>> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;เป็น Script ที่สร้างขึ้นเพื่อการสร้าง สัญญาณและnoise สำหรับการเคลื่อนที่ในแนวเชิงเส้น และการเคลื่อนที่ในเชิงมุม 
>>- __ซึ่งประกอบไปด้วย service__ ที่จะต้องรับค่า Mean และ Variance มาจาก Client โดยที่เรากำหนดใช้เป็น console ในการรส่งค่า
>>- __โดยเราต้องการ 2 Node__ ที่สร้าง Noise ออกมา เราจึงเลือกใช้ __namespace__ เพื่อเป็นการกำหนด ให้แต่ล่ะ Node มี topic ต่างๆแยกกัน เราใช้ namespace เป็น linear และ angular
>>- เมื่อรับค่าของ Mean และ Variance มาแล้วจึงนำค่าไปใส่ใน ในตัวแปร self.mean , self.variance เพื่อสำหรับการ generate ค่าความแปรปรวน ผ่าน library numpy มาเพื่อใช้ฟังชันน์ที่มีอยู่ในนั้นและทำการส่งค่า __(จะอยู่ใน code noise_generator.py line 23-25 _(crate object)_ , 39-44 _(funtion callback)_)__ 
>>- และทำการ publish ค่า noise ที่ได้ไปที่ topic __set_noise__ ผ่าน timer ที่เราสร้างขึ้น __(จะอยู่ใน code noise_generator.py line 28-33 _(crate object)_ , 46-50 _(funtion callback)_)__ 

>## Velocity Mux :
>### overview :
>>    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;เป็น Node ที่สร้างขึ้นมาเพื่อที่จะรวมสัญญาณ ความเร็วเชิงเส้นและความเร็วเชิงมุม  และทำการ publish ออกไปให้ turtlesim
>>
>>- มีการ sub 2 topic คือ /linear/noise (ซึ่งส่งค่าความแปรปรวนในการเคลื่อนที่ในเชิงเส้นมา) และ /angular/noise  (ค่าความแปรปรวนในการเคลื่อนที่เชิงมุม) และทำการนำมาเก็บค่าเข้ามาใน class __(จะอยู่ใน code velocity_controller.py line 30-35 _(crate object)_ , 41-51 _(funtion callback)_)__ 
>>
>>- มีการ publish ข้อมูลผ่าน topic /turtle1/cmd_vel โดยที่มี data type เป็นแบบ Twist โดยอาศัย timer ในการส่งข้อมูลออกไป โดยจะส่งข้อมูลออกไป ็x Hz โดยที่ x มีค่าตามที่เราใส่ argment มาให้ หรือจะมีค่า 5.0 Hz หากไม่มีการใส่ __(จะอยู่ใน code velocity_controller.py line 14-19 _(define rate)_ , 24 _(create timer)_ , 34-36 _(create object pulish)_ , 53-60 _(callback_funtion)_)__ 


>## cmakelist : เพิ่ม script ที่เราใช้ไปใน cmakelist
>>File : "fra333_lab1_11/CMakeLists.txt"
>>```cmake
>>install(PROGRAMS
>>  scripts/noise_generator.py
>>  scripts/velocity_mux.py
>>  DESTINATION lib/${PROJECT_NAME}
>>)
>>```

>## Launch File :
>### overview :
>> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;launch File คือ Script ที่ใช้ในการ run node หลายๆอันพร้อมๆกันในคราวเดียว ซึ่งประกอบไปด้วย สร้าง รัน noise_generator 2 ตัวที่มี namespace ต่างกัน , รัน turtlesim node , รัน velocity_mux , ส่งค่า argment ที่ัรบมาจากterminal ให้กับ noise_generator และ velocity_mux ,  สั่งคำสั่งผ่าน terimal ในการเรียก service ให้ noise_generator ส่งเป็น mean , variance 
>>  - สร้าง รัน noise_generator 2 ตัวที่มี namespace ต่างกัน __(จะอยู่ใน code random_walk.launch.py line 21-36)__ 
>>  - รัน turtlesim node __(จะอยู่ใน code random_walk.launch.py line 12-16)__
>>  - รัน velocity_mux __(จะอยู่ใน code random_walk.launch.py line 39-52)__
>>  - ส่งค่า argment ที่ัรับมาจากterminal ให้กับ noise_generator และ velocity_mux __(จะอยู่ใน code random_walk.launch.py line 18-20, 26,34,42)__
>>  - สั่งคำสั่งผ่าน terimal ในการเรียก service ให้ noise_generator ส่งเป็น mean , variance __(จะอยู่ใน code random_walk.launch.py line 47-62)__
>### cmakelist :
>>File : "fra333_lab1_11/CMakeLists.txt"
>>```cmake
>>install(DIRECTORY
>>  launch
>>  DESTINATION share/${PROJECT_NAME})
>>```

>## reuslt :

<p align="center">
<img src="result.png" width="90%">
</p>
<p align="center">
ภาพที่ 2 ผลลัพธ์การทำงาน
</p>
<p align="center">
<img src="rqt_graph.png" width="90%">
</p>
<p align="center">
ภาพที่ 3 rqt_graph
</p>
